# Load test of an endpoint

Load testing is a critical performance evaluation technique that simulates real-world usage patterns on a large language model endpoint. LLMeter helps measuring response times
and evaluate system throughput by varying the number of concurrent client requests. In this notebook we explore two patterns: fixed prompt and representative prompts load testing. 

For this load testing example we'll use [Amazon Bedrock](https://aws.amazon.com/bedrock/). The results will be a function of the selected [text generation model](https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html) selected, as well as the [throughput quotas](https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html) of the account.  
The result of this notebook can also be used to analyze and compare endpoints, as demonstrated in [TPOT_comparison.ipynb](TPOT_comparison.ipynb).

## Configuration

In this section we select the model we will load test.

In [ ]:
from llmeter.endpoints.bedrock import BedrockConverseStream
from llmeter.experiments import LoadTest

In [ ]:
model_id = None  # <-- Choose a Bedrock model ID from the above-linked list that supports Converse API

We now create an LLMeter endpoint to manage access to the model. We'll be using a streaming endpoint, so we will also have information about the time to first token.

In [ ]:
endpoint = BedrockConverseStream(model_id=model_id)

## Load test with fixed payload

We start by testing the endpoint by sending the same prompt consecutively, and repeat that different numbers of concurrent clients. 
Running a load test with a fixed payload allows us to compare endpoints using dimensions like _time to first token_ (TTFT), and it's also a convenient way to observe the throughput for input and output tokens, but might not be representative of the request throughput for a realistic workload.

In [ ]:
payload = endpoint.create_payload(
    "this is a test request, write a short poem, then translate it to italian",
)
payload

We define the load test by indicating the sequence of clients to test. The result will also be saved in the output path.

In [ ]:
load_test = LoadTest(
    endpoint=endpoint,
    payload=payload,
    sequence_of_clients=[1, 5, 10],
    output_path=f"outputs/{endpoint.model_id}/load_test",
    min_requests_per_run=30,
    min_requests_per_client=10
)

In [ ]:
load_test_results = await load_test.run()

## Visualize results

We can now visualize the results, using the convenience method `plot_results()`

In [ ]:
figures = load_test_results.plot_results()

If you want to compare this load test with another one, take note of the output path, that's where the result is stored. 

In [ ]:
load_test_results.output_path